In [87]:
import numpy as np
import sys
sys.path.append('../')
from utils.sparse_vector import *
from utils.vector_polynomial_isomorphism import *
from utils.minor_flattening import *

In [88]:
# Let us initialize constants and vector spaces.

N = 2*3*3
n = [2,3,3]
d = 3
indeterminates = form_indeterminates(n)
R = PolynomialRing(QQ, indeterminates)
R.inject_variables()

glN = MatrixSpace(QQ, N)

Defining x_000, x_001, x_002, x_010, x_011, x_012, x_020, x_021, x_022, x_100, x_101, x_102, x_110, x_111, x_112, x_120, x_121, x_122


In [89]:
def E_action(a, b, grad_fl):
    sumh = -1*indeterminates[b]*grad_fl[a]
    return R(sumh)

In [90]:
f = get_flattening_minors(R, n, d)
monomial_to_index = {}

# create (monomial => index) mapping for the monomials in all f_i
for fi in f:
    for mon in fi.monomials():
        if mon not in monomial_to_index:
            monomial_to_index[mon] = len(monomial_to_index)
            
# create vector space span{monomials in f_i}
dim_V_small = len(monomial_to_index)
V_small = VectorSpace(QQ, dim_V_small, sparse=True)
            
# add in monomials that are in span{monomials of Xf_i} to the (monomial => index) mapping
for fl in f:
    grad_fl = fl.gradient()
    for i in range(N):
        for j in range(N):
            E_action_fl_ij = E_action(i,j,grad_fl)
            for mon in E_action_fl_ij.monomials():
                if mon not in monomial_to_index:
                    monomial_to_index[mon] = len(monomial_to_index)

# convert f_i into vectors inside V_small
f_vec = []
for fi in f:
    fi_vec = [0]*dim_V_small
    for mon in fi.monomials():
        idx = monomial_to_index[mon]
        fi_vec[idx] = fi.coefficient(mon)
    f_vec.append(V_small(fi_vec))

In [91]:
# create W = span{f_i} inside V_small. W_perp is its orthogonal complement inside this small vector space
W = V_small.subspace(f_vec)
W_perp = W.basis_matrix().right_kernel_matrix(algorithm='default', basis='computed')

p = W.dimension()
q = W_perp.nrows()

print('p=', p, 'q=', q, 'dim_V_small=', dim_V_small)

p= 36 q= 156 dim_V_small= 192


In [92]:
# create sparse basis vectors for orthogonal complement in V_small
g_sparse = [sparsify(W_perp[k]) for k in range(q)]

# add g_i's to include into {monomials of fi}U{monomials of Xfi} by adding standard basis vectors in coordinates outside V_small
for i in range(dim_V_small, len(monomial_to_index)):
    v = [(i, 1)]
    g_sparse.append(v)

q = len(g_sparse)

In [93]:
def polyomial_to_sparsevector(p):
    ans = []
    for mon in p.monomials():
        ans.append((monomial_to_index[mon], p.coefficient(mon)))
    return ans

E_action_fl = [[0 for j in range(N)] for i in range(N)]

M_f = zero_matrix(QQ, p*q, N^2, sparse = True)
for l in range(p):
    grad_fl = f[l].gradient()
    for i in range(N):
        for j in range(N):
            E_action_fl[i][j] = polyomial_to_sparsevector(E_action(i,j,grad_fl))
    for k in range(q):
        for i in range(N):
            for j in range(N):
                M_f[(q*l) + k,(N*i) + j] = sparse_inner_product(E_action_fl[i][j], g_sparse[k])

In [94]:
rank = M_f.rank()

print('rank(M_f) =', rank)
print('dim(ker(M_f)) =', glN.dimension() - rank)

rank(M_f) = 324
dim(ker(M_f)) = 0


In [95]:
print('expected:', sum([ni^2-1 for ni in n]) + 1)

expected: 20
